In [1]:
import os
import csv
import copy
import math

import pandas as pd

C:\Users\amitg\AppData\Local\Temp\ipykernel_8280\2115673951.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'C'],
    'C': ['B', 'A'],
}

In [3]:
nodes = list(graph.keys())
node_positions = {v: i for i, v in enumerate(nodes)}
node_positions

{'A': 0, 'B': 1, 'C': 2}

In [4]:
graph_based_on_indx = {}
for k, v in graph.items():
    graph_based_on_indx[node_positions[k]] = []
    for iv in v:
        graph_based_on_indx[node_positions[k]].append(node_positions[iv])
graph_based_on_indx

{0: [1, 2], 1: [0, 2], 2: [1, 0]}

In [5]:
degree_of_nodes = {n: len(graph[n]) for n in nodes}

print("Degree of all nodes (starting from 0):")
degree_of_nodes # start from 0

Degree of all nodes (starting from 0):


{'A': 2, 'B': 2, 'C': 2}

In [6]:
degree_of_nodes_based_on_indx = {node_positions[k]: v for k, v in degree_of_nodes.items()}
degree_of_nodes_based_on_indx

{0: 2, 1: 2, 2: 2}

In [7]:
class Configuration:
    def __init__(self, val=0):
        self._val = val

    @property
    def val(self):
        return self._val

    @val.setter
    def val(self, val):
        self._val = val

    def __eq__(self, other):
        return self.val == other.val

    def __hash__(self) -> int:
        return hash(self.val)

    def __repr__(self):
        return f"<val: {self.val}>"


In [8]:
configurations = {
    tuple([Configuration(val=0) for i in range(len(nodes))])
}
# perturb each state at a time for all states in configurations and accumulate the same in the configurations for next state to perturb
for _, n in enumerate(nodes):
    node_pos = node_positions[n]
    config_copy = copy.deepcopy(configurations)
    for val in {0, 1}:
        for cc in config_copy:
            cc = list(cc)
            cc[node_pos] = Configuration(val=val)
            configurations.add(tuple(cc))

print("All possible configurations:")
len(configurations), configurations

All possible configurations:


(8,
 {(<val: 0>, <val: 0>, <val: 0>),
  (<val: 0>, <val: 0>, <val: 1>),
  (<val: 0>, <val: 1>, <val: 0>),
  (<val: 0>, <val: 1>, <val: 1>),
  (<val: 1>, <val: 0>, <val: 0>),
  (<val: 1>, <val: 0>, <val: 1>),
  (<val: 1>, <val: 1>, <val: 0>),
  (<val: 1>, <val: 1>, <val: 1>)})

In [9]:
state = [0, 0, 0, 0]

In [10]:
def check_I_lte_v_null(position, state):
    for nbr in graph_based_on_indx[position]:
        if degree_of_nodes_based_on_indx[nbr] <= degree_of_nodes_based_on_indx[position] and state[nbr] == 1:
            return False
    return True

In [11]:
def update_in_out(position):
    if state[position] == 0:
        if check_I_lte_v_null(position, state):
            state[position] = 1
    else:
        if not check_I_lte_v_null(position, state):
            state[position] = 0

    print(state)

In [12]:
def neighbor_w_lte_degree_exists(position):
    for nbr in graph_based_on_indx[position]:
        if degree_of_nodes_based_on_indx[nbr] <= degree_of_nodes_based_on_indx[position]:
            return True
    return False

In [13]:
def is_cvf(position, start_state, dest_state):
    has_nbr_w_lte_degree = neighbor_w_lte_degree_exists(position)
    if has_nbr_w_lte_degree:
        return start_state[position].val == 1
    else:
        return start_state[position].val == 0

In [15]:
for config in configurations:
    for position, _ in enumerate(config):
        perturb_value = {1, 0} - {config[position].val}
        perturb_state = list(copy.deepcopy(config))
        perturb_state[position].val = perturb_value.pop()
        perturb_state = tuple(perturb_state)
        print(f"CVF config: {config}, position: {position}, perturb state: {perturb_state}")

CVF config: (<val: 1>, <val: 0>, <val: 1>), position: 0, perturb state: (<val: 0>, <val: 0>, <val: 1>)
CVF config: (<val: 1>, <val: 0>, <val: 1>), position: 1, perturb state: (<val: 1>, <val: 1>, <val: 1>)
CVF config: (<val: 1>, <val: 0>, <val: 1>), position: 2, perturb state: (<val: 1>, <val: 0>, <val: 0>)
CVF config: (<val: 1>, <val: 1>, <val: 0>), position: 0, perturb state: (<val: 0>, <val: 1>, <val: 0>)
CVF config: (<val: 1>, <val: 1>, <val: 0>), position: 1, perturb state: (<val: 1>, <val: 0>, <val: 0>)
CVF config: (<val: 1>, <val: 1>, <val: 0>), position: 2, perturb state: (<val: 1>, <val: 1>, <val: 1>)
CVF config: (<val: 0>, <val: 1>, <val: 0>), position: 0, perturb state: (<val: 1>, <val: 1>, <val: 0>)
CVF config: (<val: 0>, <val: 1>, <val: 0>), position: 1, perturb state: (<val: 0>, <val: 0>, <val: 0>)
CVF config: (<val: 0>, <val: 1>, <val: 0>), position: 2, perturb state: (<val: 0>, <val: 1>, <val: 1>)
CVF config: (<val: 0>, <val: 0>, <val: 0>), position: 0, perturb state: (